# Polarizations over the sky

This notebook visualizes in 3D the effect of gravitational waves from a continuous source (e.g., an asymmetric neutron star, or a compact binary at it very early moments of inspiral) emitting in the primary $\ell=|m|=2$ angular mode of radiation.

The source (not shown) will be placed at the center of a sphere representing the sky infinitely far away from the source.
On the sphere, we will distribute a number of rings of freely falling particles, which will serve to illustrate the effect of the gravitational waves as a function of inclination and azimuthal angle around the source.

We will find that points at the poles will receive circularly polarized gravitational waves, points at the equator receive linearly polarized waves, and intermediate points receive waves with varying degrees of ellipticity as a function of the inclination (polar angle).

In [1]:
# first cell
from mayavi import mlab
mlab.init_notebook()

Notebook initialized with ipy backend.


In [12]:
# second cell
# mlab.test_plot3d()

In [3]:
from pylab import *
import seaborn as sns
from mayavi import mlab

import imageio
from tqdm.notebook import tqdm
import os

from matplotlib.patches import Circle, Wedge, Polygon
from utils.plots import EllipticalWedge

sns.set(context='notebook', palette='colorblind', style='ticks',
        font='serif', font_scale=1.5)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

plt.rcParams["text.usetex"] = "true"

In [4]:
from pyspherical import spin_spherical_harmonic

In [5]:
# define source-centered Cartesian frame {X, Y, Z} based on which to 
# define Ylm's
X = array([1, 0, 0])
Y = array([0, 1, 0])
Z = array([0, 0, 1])

# the wave propagation vector will be
def wave_nhat(theta, phi):
    n = [sin(theta)*cos(phi), sin(theta)*sin(phi), cos(theta)]
    return array(n)

# define polarization axes X and Y following LIGO convention
# https://lscsoft.docs.ligo.org/lalsuite/lalsimulation/group___l_a_l_simulation__h.html
# https://lscsoft.docs.ligo.org/lalsuite/lalsimulation/group__lalsimulation__inspiral.html
def wave_xy(theta=None, phi=None, nhat=None):
    if nhat is None:
        nhat = wave_nhat(theta, phi)
    # the x vector will be n x Z
    if nhat[2] == 1:
        # theta = 0 so there's ambiguity about frame orientation,
        # disambiguate manually
        x = array([cos(phi), sin(phi), 0])
    else:
        x = cross(nhat, Z)
    # the y vector is n x x
    y = cross(nhat, x)
    return x/linalg.norm(x), y/linalg.norm(y)

def get_ring(theta=None, phi=None, nhat=None, npoints=16, r=0.1, complete=True):
    # distribute `npoints` points in a circle around the sky location 
    # defined by theta and phi, with radius `r` around the center
    n = nhat if nhat is not None else wave_nhat(theta, phi)
    x, y = wave_xy(theta, phi, n)
    # compute angular separation between points
    angle = 2*pi / npoints
    # first point will be anchored in the direction of the waveframe x
    points = [r*x]
    # now rotate that initial point `npoints` times around nhat
    # to do that, first define a rotation matrix around n
    # (this is simply a composition of matrices rotating aroung X, Y, Z,
    # but let's code up the resulting matrix---lifted from Wikipedia---
    # directly to make this more efficient
    costh, sinth = cos(angle), sin(angle)
    p = 1 - costh
    R = array([[costh + n[0]*n[0]*p,      n[0]*n[1]*p - n[2]*sinth, n[0]*n[2]*p + n[1]*sinth],
                [n[0]*n[1]*p + n[2]*sinth, costh + n[1]*n[1]*p,      n[1]*n[2]*p - n[0]*sinth],
                [n[0]*n[2]*p - n[1]*sinth, n[2]*n[1]*p + n[0]*sinth, costh + n[2]*n[2]*p]])
    # we can now apply the rotation iteratively
    for i in range(1, npoints):
        points.append(dot(R, points[i-1]))
    if complete:
        points.append(points[0])
    return array(points)

def get_epec(theta=None, phi=None, nhat=None):
    x, y = wave_xy(theta, phi, nhat)
    ep = outer(x, x) - outer(y, y)
    ec = outer(x, y) + outer(y, x)
    return ep, ec    

In [6]:
# functions to lay down Fibonacci grid on the sphere

def cartesian_to_spherical(xyz, north_polar=True):
    """ Efficiently convert points from Carstesian to spherical coordinates.

    https://stackoverflow.com/questions/4116658/faster-numpy-cartesian-to-spherical-coordinate-conversion
    """
    ptsnew = np.zeros(xyz.shape)
    xy = xyz[:,0]**2 + xyz[:,1]**2
    # radius
    ptsnew[:,0] = np.sqrt(xy + xyz[:,2]**2)
    if north_polar:
        # elevation (polar) angle defined from Z-axis down
        ptsnew[:,1] = np.arctan2(np.sqrt(xy), xyz[:,2])
    else:
        # elevation (polar) angle defined from XY-plane up
        ptsnew[:,1] = np.arctan2(xyz[:,2], np.sqrt(xy))
    # azimuthal angle from x-axis
    ptsnew[:,2] = np.arctan2(xyz[:,1], xyz[:,0])
    return ptsnew


def fibonacci_sphere(samples, randomize=False, spherical=True, **kwargs):
    """ Create a Fibonacci grid on the unit sphere.

    https://stackoverflow.com/questions/9600801/evenly-distributing-n-points-on-a-sphere

    Arguments
    ---------
    samples : int
        number of points to draw
    randomize : bool

    Returns
    -------
    points : array
        [x, y, z] or [r, theta, phi] or [r, np.pi-theta, phi]
    """
    rnd = 1.
    if randomize:
        rnd = random.random() * samples
    points = []
    offset = 2./samples
    increment = np.pi * (3. - np.sqrt(5.));
    for i in range(samples):
        y = ((i * offset) - 1) + (offset / 2);
        r = np.sqrt(1 - pow(y, 2))
        phi = ((i + rnd) % samples) * increment
        x = np.cos(phi) * r
        z = np.sin(phi) * r
        points.append([x,y,z])
    points = np.array(points)
    if spherical:
        points = cartesian_to_spherical(points, **kwargs)
    return points

In [7]:
# fig = plt.figure()
# ax = fig.add_subplot(projection='3d')

# th, phi = pi/2, 3*pi/2
# ps = get_ring(th, phi)
# n = wave_nhat(th, phi)
# ps += n

# ax.scatter(ps[:,0], ps[:,1], ps[:,2])

# na = vstack([zeros(3), n])
# ax.plot(na[:,0], na[:,1], na[:,2])

In [8]:
import matplotlib.colors as mcolors

In [9]:
def sphere_snap(wt, nsphere=56, ncircle=16, rcircle=0.5, rsphere=5, A=1, background=True,
                cmap='plasma', sphere_color=(0.2, 0.2, 0.2)):
    # let's pick a number of points in the sphere to serve as centers
    # to our little rings (these are just nhats in spherical coordinates)
    nhats_sph = fibonacci_sphere(nsphere)

    # let's produce a ring around each of these sky locations
    rings = [get_ring(theta, phi, npoints=ncircle, r=rcircle) for _, theta, phi in nhats_sph]


    nhats_xyz = rsphere*fibonacci_sphere(nsphere, spherical=False)

    el, em = 2, 2

#     fig = plt.figure(dpi=150)
#     ax = fig.add_subplot(projection='3d')
    
    cmap = matplotlib.cm.get_cmap(cmap)
    norm = matplotlib.colors.Normalize(vmin=-A, vmax=A)


    
#     for a in [X,Y,Z]:
#         p = vstack([zeros(3), a])
#         plot(p[:,0], p[:,1], p[:,2], c='gray')
        
#     # Make sphere data
#     u = np.linspace(0, 2 * np.pi, 100)
#     v = np.linspace(0, np.pi, 100)
#     x = 0.95 *rsphere * np.outer(np.cos(u), np.sin(v))
#     y = 0.95 *rsphere * np.outer(np.sin(u), np.sin(v))
#     z = 0.95 *rsphere * np.outer(np.ones(np.size(u)), np.cos(v))

#     # Plot the surface
#     ax.plot_surface(x, y, z, color='b', lw=0)

    mlab.points3d(nhats_xyz[:,0], nhats_xyz[:,1], nhats_xyz[:,2], mode='point')#scale_factor=0.5)
    
    for n, nsph, ring in zip(nhats_xyz, nhats_sph, rings):
        _, theta, phi = nsph

        h = A * spin_spherical_harmonic(-2, el, em, theta, phi)*exp(-1j*em*wt) + \
            (-1)**el * A * spin_spherical_harmonic(-2, el, -em, theta, phi)*exp(1j*em*wt)
        ep, ec = get_epec(theta, phi)

        dx = dot(h.real*ep - h.imag*ec, ring.T).T

        new_ring = ring + dx
        ps = n + new_ring
        s = mlab.points3d(ps[:,0], ps[:,1], ps[:,2], color=cmap(abs(h))[:-1], scale_factor=0.06)#, lw=1)
    if background:
        [phi, theta] = np.mgrid[0:2 * np.pi:56j, 0:np.pi:56j]
        x = np.cos(phi) * np.sin(theta)
        y = np.sin(phi) * np.sin(theta)
        z = np.cos(theta)

        r = rsphere
        # cc = mcolors.to_rgb("midnightblue")
        cc = sphere_color
        # cc = tuple(np.array(cc) / linalg.norm(cc))
        mlab.mesh(r * x, r * y, r * z, opacity=0.5, color=cc)
    return s

In [10]:
fig = mlab.figure(size=(1024, 1024))
fig.scene.background = (1, 1, 1) 
s = sphere_snap(0, nsphere=1024, rcircle=0.1)
s

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x04\x00\x00\x00\x04\x00\x08\x02\x00\x00\x00\xf0\x7f\…

In [11]:
colors = {
    'dark': (0.2, 0.2, 0.2),
    # 'light': (0.8, 0.8, 0.8),
    # 'medium': (0.5, 0.5, 0.5),
}
for color, rgb in colors.items():
    for cmap in ['viridis', 'plasma']:
        fig = mlab.figure()#size=(1024, 1024))
        fig.scene.background = (1, 1, 1) 
        nsnaps = 50
        basedir = f'sphere_{cmap}_{color}'
        if not os.path.exists(basedir):
            os.mkdir(basedir)
        png_path = os.path.join(basedir, 'snap{}.png')
        for i, wt in enumerate(tqdm(linspace(0, 2*pi, nsnaps))):
            fig = mlab.figure(size=(1024, 1024))
            fig.scene.background = (1, 1, 1) 
            s = sphere_snap(wt, nsphere=512, rcircle=0.2, cmap=cmap, sphere_color=rgb)
            mlab.savefig(png_path.format(i))

        images = []
        for i in range(nsnaps):
            file_path = png_path.format(i)
            images.append(imageio.imread(file_path))
        imageio.mimsave(f'{basedir}/movie.gif', images, fps=12)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

### License

Copyright © 2023 Maximiliano Isi

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the “Software”), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.